In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mutual_info_score,  roc_curve, auc, confusion_matrix, classification_report, roc_auc_score, precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV, cross_val_predict, cross_val_score
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm

# read data
#file path: RFs_PET1_bin0.25\6-4ADASYN-1\post-Combat (or post-Limma)\lasso (or mRMR or JMI or JMIM)
train_data = pd.read_csv(r'C:\Users\37427\Desktop\github\FS-ML\RFs_PET1_bin0.25\6-4ADASYN-1\post-Combat\JMI\train.csv')
test_data = pd.read_csv(r'C:\Users\37427\Desktop\github\FS-ML\RFs_PET1_bin0.25\6-4ADASYN-1\post-Combat\JMI\test.csv')

X_train = train_data.iloc[:, 1:]  
y_train = train_data.iloc[:, 0]  
X_test = test_data.iloc[:, 1:]   
y_test = test_data.iloc[:, 0]    

#Classifier --- Adaptive Boosting
np.random.seed(0)
param_grid = {
    'n_estimators': [50, 100, 200],  
    'learning_rate': [0.01, 0.1, 0.2], 
    'algorithm': ['SAMME', 'SAMME.R'], 
}
ada_clf = AdaBoostClassifier()
grid_search = GridSearchCV(estimator=ada_clf, param_grid=param_grid,cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

y_train_pred_cv = cross_val_predict(best_model, X_train, y_train, cv=5)
y_train_pred_proba_cv = cross_val_predict(best_model, X_train, y_train, cv=5, method='predict_proba')[:, 1]
train_precision_cv, train_recall_cv, train_f1_cv, _ = precision_recall_fscore_support(y_train, y_train_pred_cv, average='binary')
y_test_pred = best_model.predict(X_test)
y_test_pred_proba = best_model.predict_proba(X_test)[:, 1]
test_precision, test_recall, test_f1, _ = precision_recall_fscore_support(y_test, y_test_pred, average='binary')

train_accuracy_cv = cross_val_score(best_model, X_train, y_train, cv=5, scoring='accuracy').mean()
test_accuracy = accuracy_score(y_test, y_test_pred)
conf_mat_train_cv = confusion_matrix(y_train, y_train_pred_cv)
conf_mat_test = confusion_matrix(y_test, y_test_pred)
sensitivity_train = conf_mat_train_cv[1, 1] / (conf_mat_train_cv[1, 1] + conf_mat_train_cv[1, 0])  # TP / (TP + FN)
sensitivity_test = conf_mat_test[1, 1] / (conf_mat_test[1, 1] + conf_mat_test[1, 0])  # TP / (TP + FN)
specificity_train = conf_mat_train_cv[0, 0] / (conf_mat_train_cv[0, 0] + conf_mat_train_cv[1, 0])
specificity_test = conf_mat_test[0, 0] / (conf_mat_test[0, 0] + conf_mat_test[1, 0])
ppv_train = conf_mat_train_cv[1, 1] / (conf_mat_train_cv[1, 1] + conf_mat_train_cv[0, 1])
npv_train = conf_mat_train_cv[0, 0] / (conf_mat_train_cv[0, 0] + conf_mat_train_cv[1, 0])
ppv_test = conf_mat_test[1, 1] / (conf_mat_test[1, 1] + conf_mat_test[0, 1])
npv_test = conf_mat_test[0, 0] / (conf_mat_test[0, 0] + conf_mat_test[1, 0])
train_precision = precision_score(y_train, y_train_pred_cv)
test_precision = precision_score(y_test, y_test_pred)
train_recall = recall_score(y_train, y_train_pred_cv)
test_recall = recall_score(y_test, y_test_pred)
train_f1 = f1_score(y_train, y_train_pred_cv)
test_f1 = f1_score(y_test, y_test_pred)
print(f"Sensitivity (Cross-Validation): {sensitivity_train:.4f}")
print(f"Specificity (Cross-Validation): {specificity_train:.4f}")
print(f"PPV (Cross-Validation): {ppv_train:.4f}")
print(f"NPV (Cross-Validation): {npv_train:.4f}")
print(f"Precision (Cross-Validation): {train_precision:.4f}")
print(f"Recall (Cross-Validation): {train_recall:.4f}")
print(f"F1 Score (Cross-Validation): {train_f1:.4f}")
print(f"Train Accuracy (Cross-Validation): {train_accuracy_cv:.4f}")
print(f"Sensitivity (Test): {sensitivity_test:.4f}")
print(f"Specificity (Test): {specificity_test:.4f}")
print(f"PPV (Test): {ppv_test:.4f}")
print(f"NPV (Test): {npv_test:.4f}")
print(f"Precision (Test): {test_precision:.4f}")
print(f"Recall (Test): {test_recall:.4f}")
print(f"F1 Score (Test): {test_f1:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

fpr_train, tpr_train, _ = roc_curve(y_train, y_train_pred_proba_cv)
fpr_test, tpr_test, _ = roc_curve(y_test, y_test_pred_proba)
np.random.seed(0)
def bootstrap_roc_auc(y_true, y_scores, n_bootstraps=1000):
    auc_scores = np.zeros(n_bootstraps)
    for i in range(n_bootstraps):
        indices = np.random.choice(len(y_true), size=len(y_true))
        auc_scores[i] = roc_auc_score(y_true[indices], y_scores[indices])
    return auc_scores

auc_scores_train = bootstrap_roc_auc(y_train, y_train_pred_proba_cv)
auc_train_mean = np.mean(auc_scores_train)
auc_train_std = np.std(auc_scores_train)
auc_train_lower = norm.ppf(0.025, loc=auc_train_mean, scale=auc_train_std)
auc_train_upper = norm.ppf(0.975, loc=auc_train_mean, scale=auc_train_std)
print(f"Cross-Validation AUC: {auc_train_mean:.4f} (95% CI: {auc_train_lower:.4f}, {auc_train_upper:.4f})")

auc_scores_test = bootstrap_roc_auc(y_test, y_test_pred_proba)
auc_test_mean = np.mean(auc_scores_test)
auc_test_std = np.std(auc_scores_test)
auc_test_lower = norm.ppf(0.025, loc=auc_test_mean, scale=auc_test_std)
auc_test_upper = norm.ppf(0.975, loc=auc_test_mean, scale=auc_test_std)
print(f"Test AUC: {auc_test_mean:.4f} (95% CI: {auc_test_lower:.4f}, {auc_test_upper:.4f})")

Best parameters: {'algorithm': 'SAMME.R', 'learning_rate': 0.2, 'n_estimators': 200}
Sensitivity (Cross-Validation): 0.6804
Specificity (Cross-Validation): 0.7207
PPV (Cross-Validation): 0.6875
NPV (Cross-Validation): 0.7207
Precision (Cross-Validation): 0.6875
Recall (Cross-Validation): 0.6804
F1 Score (Cross-Validation): 0.6839
Train Accuracy (Cross-Validation): 0.7053
Sensitivity (Test): 0.6316
Specificity (Test): 0.7846
PPV (Test): 0.5333
NPV (Test): 0.7846
Precision (Test): 0.5333
Recall (Test): 0.6316
F1 Score (Test): 0.5783
Test Accuracy: 0.6818
Cross-Validation AUC: 0.7431 (95% CI: 0.6773, 0.8090)
Test AUC: 0.7466 (95% CI: 0.6556, 0.8375)
